# Check whether the model is working or not

In [ ]:
import onnxruntime
import numpy as np
import cv2

def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (96, 96))
    image = image.astype(np.float32)
    image /= 255.0  # Normalize the image to [0, 1]
    image = np.transpose(image, (2, 0, 1))  # Channels-first format (C,H,W)
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    return image

def main():
    model_path = "/home/muhammad/.insightface/models/buffalo_l/genderage.onnx"

    providers = ['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']
    session = onnxruntime.InferenceSession(model_path, providers=providers)
    image_url = "image.jpg"
    image = preprocess_image(image_url)

    input_name = session.get_inputs()[0].name
    output_name = session.get_outputs()[0].name
    pred = session.run([output_name], {input_name: image})
    gender = np.argmax(pred[0][0][:2])
    age = int(np.round(pred[0][0][2]*100))
    print(gender)
    print(age)

main()


# Convert the model to tflite

In [ ]:
import onnx
from onnx_tf.backend import prepare
import tensorflow as tf
 
onnx_model = onnx.load("/home/muhammad/.insightface/models/buffalo_l/genderage.onnx")
tf_rep = prepare(onnx_model)
tf_rep.export_graph("output/model.pb")

# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model("output/model") # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

# checking the tflite output

In [ ]:
import numpy as np
import tensorflow as tf
import cv2

def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (96, 96))
    image = image.astype(np.float32)
    image /= 255.0  # Normalize the image to [0, 1]
    image = np.transpose(image, (2, 0, 1))  # Channels-first format (C,H,W)
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    return image

# Step 1: Load the TFLite Model
tflite_model_path = 'model.tflite'

# Step 2: Initialize the Interpreter
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

# Step 3: Prepare Input Data
# Replace this with your own input data
image_url = "image.jpg"
image = preprocess_image(image_url)
# input_data = np.random.rand(1, input_height, input_width, input_channels).astype(np.float32)

# Step 4: Run Inference
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Set input tensor
interpreter.set_tensor(input_details[0]['index'], image)

# Run inference
interpreter.invoke()

# Step 5: Process Output
output_data = interpreter.get_tensor(output_details[0]['index'])

# Process the output as needed to get the final predictions
# For example, if it's a classification model, you might apply softmax to get probabilities.
# For regression models, you might simply use the output as the predictions.
gender = np.argmax(output_data[0][:2])
age = int(np.round(output_data[0][2]*100))
print(gender)
print(age)